# Stock NeurIPS2018 Part 3. Backtest
This series is a reproduction of paper *the process in the paper Practical Deep Reinforcement Learning Approach for Stock Trading*. 

This is the third and last part of the NeurIPS2018 series, introducing how to use use the agents we trained to do backtest, and compare with baselines such as Mean Variance Optimization and DJIA index.

Other demos can be found at the repo of [FinRL-Tutorials]((https://github.com/AI4Finance-Foundation/FinRL-Tutorials)).

# Part 1. Install Packages

In [1]:
# ## install required packages
# !pip install swig
# !pip install wrds
# !pip install pyportfolioopt
# ## install finrl library
# !pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

In [2]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3

%matplotlib inline
from finrl.config import INDICATORS

# Part 2. Backtesting

To backtest the agents, upload trade_data.csv in the same directory of this notebook. For Colab users, just upload trade_data.csv to the default directory.

In [59]:
train = pd.read_csv('train_data.csv')
trade = pd.read_csv('trade_data.csv')

# If you are not using the data generated from part 1 of this tutorial, make sure 
# it has the columns and index in the form that could be make into the environment. 
# Then you can comment and skip the following lines.
train = train.set_index(train.columns[0])
train.index.names = ['']
trade = trade.set_index(trade.columns[0])
trade.index.names = ['']

Then, upload the trained agent to the same directory, and set the corresponding variable to True.

In [60]:
if_using_a2c = True
if_using_ddpg = True
if_using_ppo = False
if_using_td3 = False
if_using_sac = False

Load the agents

In [63]:
trained_a2c =  A2C. load("./trained_models/agent_a2c") if if_using_a2c else None
trained_ddpg = DDPG.load("./trained_models/agent_ddpg") if if_using_ddpg else None
trained_ppo =  PPO. load("./trained_models/agent_ppo") if if_using_ppo else None
trained_td3 =  TD3. load("./trained_models/agent_td3") if if_using_td3 else None
trained_sac =  SAC. load("./trained_models/agent_sac") if if_using_sac else None

### Trading (Out-of-sample Performance)

We update periodically in order to take full advantage of the data, e.g., retrain quarterly, monthly or weekly. We also tune the parameters along the way, in this notebook we use the in-sample data from 2009-01 to 2020-07 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [64]:
stock_dimension = len(trade.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 29, State Space: 291


In [65]:
# trade

In [66]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

In [67]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [68]:
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_a2c, 
    environment = e_trade_gym) if if_using_a2c else (None, None)

hit end!


In [69]:
df_account_value_ddpg, df_actions_ddpg = DRLAgent.DRL_prediction(
    model=trained_ddpg, 
    environment = e_trade_gym) if if_using_ddpg else (None, None)

hit end!


In [70]:
df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
    model=trained_ppo, 
    environment = e_trade_gym) if if_using_ppo else (None, None)

In [71]:
df_account_value_td3, df_actions_td3 = DRLAgent.DRL_prediction(
    model=trained_td3, 
    environment = e_trade_gym) if if_using_td3 else (None, None)

In [72]:
df_account_value_sac, df_actions_sac = DRLAgent.DRL_prediction(
    model=trained_sac, 
    environment = e_trade_gym) if if_using_sac else (None, None)

# Part 3: Mean Variance Optimization

Mean Variance optimization is a very classic strategy in portfolio management. Here, we go through the whole process to do the mean variance optimization and add it as a baseline to compare.

First, process dataframe to the form for MVO weight calculation.

In [73]:
def process_df_for_mvo(df):
  df = df.sort_values(['date','tic'],ignore_index=True)[['date','tic','close']]
  fst = df
  fst = fst.iloc[0:stock_dimension, :]
  tic = fst['tic'].tolist()

  mvo = pd.DataFrame()

  for k in range(len(tic)):
    mvo[tic[k]] = 0

  for i in range(df.shape[0]//stock_dimension):
    n = df
    n = n.iloc[i * stock_dimension:(i+1) * stock_dimension, :]
    date = n['date'][i*stock_dimension]
    mvo.loc[date] = n['close'].tolist()
  
  return mvo

### Helper functions for mean returns and variance-covariance matrix

In [74]:
# Codes in this section partially refer to Dr G A Vijayalakshmi Pai

# https://www.kaggle.com/code/vijipai/lesson-5-mean-variance-optimization-of-portfolios/notebook

def StockReturnsComputing(StockPrice, Rows, Columns): 
  import numpy as np 
  StockReturn = np.zeros([Rows-1, Columns]) 
  for j in range(Columns):        # j: Assets 
    for i in range(Rows-1):     # i: Daily Prices 
      StockReturn[i,j]=((StockPrice[i+1, j]-StockPrice[i,j])/StockPrice[i,j])* 100 
      
  return StockReturn

### Calculate the weights for mean-variance

In [75]:
StockData = process_df_for_mvo(train)
TradeData = process_df_for_mvo(trade)

TradeData.to_numpy()

array([[141.404, 203.846, 170.065, ...,  49.275,  43.724, 133.839],
       [137.925, 201.483, 168.814, ...,  49.456,  43.621, 132.55 ],
       [139.878, 201.883, 170.867, ...,  49.465,  43.995, 133.419],
       ...,
       [149.4  , 237.62 , 174.494, ...,  37.903,  36.21 , 141.51 ],
       [146.71 , 233.66 , 173.607, ...,  38.109,  35.8  , 141.888],
       [147.92 , 234.45 , 172.66 , ...,  38.247,  35.39 , 140.863]])

In [76]:
#compute asset returns
arStockPrices = np.asarray(StockData)
[Rows, Cols]=arStockPrices.shape
arReturns = StockReturnsComputing(arStockPrices, Rows, Cols)

#compute mean returns and variance covariance matrix of returns
meanReturns = np.mean(arReturns, axis = 0)
covReturns = np.cov(arReturns, rowvar=False)
 
#set precision for printing results
np.set_printoptions(precision=3, suppress = True)

#display mean returns and variance-covariance matrix of returns
print('Mean returns of assets in k-portfolio 1\n', meanReturns)
print('Variance-Covariance matrix of returns\n', covReturns)

Mean returns of assets in k-portfolio 1
 [0.12  0.065 0.07  0.08  0.068 0.116 0.051 0.039 0.073 0.049 0.102 0.078
 0.024 0.06  0.048 0.072 0.039 0.064 0.045 0.048 0.096 0.092 0.046 0.058
 0.104 0.094 0.043 0.034 0.048]
Variance-Covariance matrix of returns
 [[3.14  1.017 1.294 1.529 1.374 1.696 1.379 1.13  1.092 1.377 1.165 1.224
  1.035 1.526 0.693 1.272 0.667 0.819 1.032 0.737 1.579 1.145 0.672 0.863
  1.146 1.34  0.526 0.909 0.635]
 [1.017 2.406 1.076 1.016 1.067 1.218 1.027 0.967 0.938 1.128 0.984 1.032
  0.851 1.155 0.892 1.143 0.643 0.668 0.909 1.074 1.092 0.879 0.727 0.855
  1.147 1.058 0.629 1.027 0.648]
 [1.294 1.076 3.307 2.486 1.912 1.575 1.435 1.926 1.721 2.24  1.382 1.859
  1.342 1.534 0.866 2.37  0.994 1.082 1.384 0.959 1.362 1.424 0.73  1.509
  1.416 1.802 0.75  1.223 0.586]
 [1.529 1.016 2.486 4.984 2.129 1.698 1.518 2.153 1.845 2.214 1.537 2.132
  1.536 1.706 0.911 2.308 1.123 1.201 1.47  0.947 1.444 1.583 0.752 1.613
  1.483 1.722 0.738 1.374 0.609]
 [1.374 1.067 1.91

### Use PyPortfolioOpt

In [77]:
from pypfopt.efficient_frontier import EfficientFrontier

ef_mean = EfficientFrontier(meanReturns, covReturns, weight_bounds=(0, 0.5))
raw_weights_mean = ef_mean.max_sharpe()
cleaned_weights_mean = ef_mean.clean_weights()
mvo_weights = np.array([1000000 * cleaned_weights_mean[i] for i in range(29)])
mvo_weights

array([288130.,      0.,      0.,      0.,      0.,  69330.,      0.,
            0.,      0.,      0., 316160.,      0.,      0.,      0.,
            0.,      0.,      0.,      0.,      0.,      0.,      0.,
        66290.,      0.,      0., 237410.,  22690.,      0.,      0.,
            0.])

In [78]:
LastPrice = np.array([1/p for p in StockData.tail(1).to_numpy()[0]])
Initial_Portfolio = np.multiply(mvo_weights, LastPrice)
Initial_Portfolio

array([2054.193,    0.   ,    0.   ,    0.   ,    0.   ,  255.623,
          0.   ,    0.   ,    0.   ,    0.   ,  998.633,    0.   ,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.   ,
          0.   ,    0.   ,    0.   ,  463.5  ,    0.   ,    0.   ,
        619.578,  103.016,    0.   ,    0.   ,    0.   ])

In [79]:
TradeData @ Initial_Portfolio

2021-10-01    1.007573e+06
2021-10-04    9.921956e+05
2021-10-05    1.004252e+06
2021-10-06    1.008231e+06
2021-10-07    1.025692e+06
                  ...     
2023-02-21    1.021317e+06
2023-02-22    1.021328e+06
2023-02-23    1.027384e+06
2023-02-24    1.013146e+06
2023-02-27    1.014912e+06
Length: 353, dtype: float64

In [80]:
Portfolio_Assets = TradeData @ Initial_Portfolio
MVO_result = pd.DataFrame(Portfolio_Assets, columns=["Mean Var"])
# MVO_result

# Part 4: DJIA index

Add DJIA index as a baseline to compare with.

In [81]:
from datetime import datetime

TRAIN_START_DATE = '2010-01-01'
TRAIN_END_DATE = '2021-10-01'
TRADE_START_DATE = '2021-10-01'
TRADE_END_DATE = '2023-03-01'
TRADE_END_DATE = datetime.today().strftime('%Y-%m-%d')

In [82]:
df_dji = YahooDownloader(start_date = TRADE_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = ['^DJI']).fetch_data()
# df_dji

[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (396, 8)


In [83]:
df_dji = df_dji[['date','close']]
fst_day = df_dji['close'][0]
dji = pd.merge(df_dji['date'], df_dji['close'].div(fst_day).mul(1000000), 
               how='outer', left_index=True, right_index=True).set_index('date')
# dji

<a id='4'></a>
# Part 5: Backtesting Results
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

In [84]:
df_result_a2c = df_account_value_a2c.set_index(df_account_value_a2c.columns[0]) if if_using_a2c else None
df_result_ddpg = df_account_value_ddpg.set_index(df_account_value_ddpg.columns[0]) if if_using_ddpg else None
df_result_ppo = df_account_value_ppo.set_index(df_account_value_ppo.columns[0]) if if_using_ppo else None
df_result_td3 = df_account_value_td3.set_index(df_account_value_td3.columns[0]) if if_using_td3 else None
df_result_sac = df_account_value_sac.set_index(df_account_value_sac.columns[0]) if if_using_sac else None

result = pd.DataFrame()
if if_using_a2c: result = pd.merge(result, df_result_a2c, how='outer', left_index=True, right_index=True)
if if_using_ddpg: result = pd.merge(result, df_result_ddpg, how='outer', left_index=True, right_index=True)
if if_using_ppo: result = pd.merge(result, df_result_ppo, how='outer', left_index=True, right_index=True)
if if_using_td3: result = pd.merge(result, df_result_td3, how='outer', left_index=True, right_index=True)
if if_using_sac: result = pd.merge(result, df_result_sac, how='outer', left_index=True, right_index=True)
result = pd.merge(result, MVO_result, how='outer', left_index=True, right_index=True)
result = pd.merge(result, dji, how='outer', left_index=True, right_index=True).fillna(method='bfill')

In [85]:
col_name = []
col_name.append('A2C') if if_using_a2c else None
col_name.append('DDPG') if if_using_ddpg else None
col_name.append('PPO') if if_using_ppo else None
col_name.append('TD3') if if_using_td3 else None
col_name.append('SAC') if if_using_sac else None
col_name.append('Mean Var')
col_name.append('djia') 
result.columns = col_name

In [88]:
result

,A2C,DDPG,Mean Var,djia
date,,,,
2021-10-01,1.000000e+06,1.000000e+06,1.007573e+06,1.000000e+06
2021-10-04,9.977105e+05,9.980525e+05,9.921956e+05,9.905746e+05
2021-10-05,1.002990e+06,1.000684e+06,1.004252e+06,9.996566e+05
2021-10-06,1.006111e+06,1.002122e+06,1.008231e+06,1.002637e+06
2021-10-07,1.014819e+06,1.010846e+06,1.025692e+06,1.012483e+06
...,...,...,...,...
2023-04-24,NaN,NaN,NaN,9.868596e+05
2023-04-25,NaN,NaN,NaN,9.768216e+05
2023-04-26,NaN,NaN,NaN,9.701516e+05


Now, everything is ready, we can plot the backtest result.

In [89]:
# plt.rcParams["figure.figsize"] = (15,5)
# plt.figure()
# result.plot()

In [94]:
import plotly.graph_objects as go
f = go.Figure()
for i in result.columns:
        
    f.add_trace(
        go.Scatter(
            x = result.index,
            y = result[i],
            name = i,
        )
    )
f